In [1]:
# package imports
import os
os.environ['PROJ_LIB'] = r'C:\ProgramData\Anaconda3\pkgs\proj-7.1.1-h7d85306_3\Library\share'

#basics
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
import json
import re
from collections import defaultdict
from datetime import datetime

#misc
import gc
import time
import warnings


#viz
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.gridspec as gridspec 
import matplotlib.gridspec as gridspec 



# Business Data

In [2]:
data = []
with open("./dataset/yelp_academic_dataset_business.json",encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
business = pd.DataFrame(data)

In [3]:
business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [122]:
business.head(1)

,business_id,name,city,state,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6iYb2HFDywm3zjuRg0shjw,Oskar Blues Taproom,Boulder,CO,40.017544,-105.283348,4.0,86,1,"{'RestaurantsTableService': 'True', 'WiFi': 'u'free'', 'BikeParking': 'True', 'BusinessParking': '{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}', 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'WheelchairAccessible': 'True', 'Caters': 'True', 'OutdoorSeating': 'True', 'RestaurantsGoodForGroups': 'True', 'HappyHour': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsPriceRange2': '2', 'Ambience': '{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}', 'HasTV': 'True', 'Alcohol': ''beer_and_wine'', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': False, 'dinner': False, 'brunch': False, 'breakfast': False}', 'DogsAllowed': 'False', 'RestaurantsTakeOut': 'True', 'NoiseLevel': 'u'average'', 'RestaurantsAttire': ''casual'', 'RestaurantsDelivery': 'None'}","Gastropubs, Food, Beer Gardens, Restaurants, Bars, American (Traditional), Beer Bar, Nightlife, Breweries","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0', 'Wednesday': '11:0-23:0', 'Thursday': '11:0-23:0', 'Friday': '11:0-23:0', 'Saturday': '11:0-23:0', 'Sunday': '11:0-23:0'}"


In [8]:
business.shape

(160585, 14)

In [9]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160585 entries, 0 to 160584
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   160585 non-null  object 
 1   name          160585 non-null  object 
 2   address       160585 non-null  object 
 3   city          160585 non-null  object 
 4   state         160585 non-null  object 
 5   postal_code   160585 non-null  object 
 6   latitude      160585 non-null  float64
 7   longitude     160585 non-null  float64
 8   stars         160585 non-null  float64
 9   review_count  160585 non-null  int64  
 10  is_open       160585 non-null  int64  
 11  attributes    145593 non-null  object 
 12  categories    160470 non-null  object 
 13  hours         133244 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 17.2+ MB


# Data Cleaning

0. remove rows with missing values
1. filter out only restaurants
2. filter `open=1` 
3. clean categories
4. clean attribute
5. clean hours

`Series.str.contains(pat, case=True, flags=0, na=None, regex=True)`:
- Fill value for missing values. The default depends on **dtype of the array**. For **object-dtype**, `numpy.nan` is used

## filter ONLY restaurant+open=1

In [4]:
mask = business.categories.str.contains('Restaurant') & (business.is_open == 1)
rests = business[mask]
rests = rests.dropna()

In [4]:
rests.shape

(28684, 14)

In [5]:
print(f"All business number:{business.shape[0]}")
print(f"Open restaurants business number:{rests.shape[0]}")

All business number:160585
Open restaurants business number:28684


## Filter out restaurants in TOP 5 cities

In [7]:
rests.city.value_counts(ascending=False)[:10]

Portland     2848
Vancouver    2675
Austin       2608
Orlando      2264
Atlanta      2263
Columbus     1566
Boston       1464
Richmond      494
Kissimmee     445
Cambridge     404
Name: city, dtype: int64

In [5]:
# top5_city
top5_city = rests.city.value_counts(ascending=False)[:5].index
rests5 = rests[rests.city.isin(top5_city)]
rests5 = rests5.reset_index(drop=True)

In [6]:
rests5.shape #28684-->12658 restaurants

(12658, 14)

In [6]:
# store metadata about restaurants first
business_info = rests5[["business_id","name","city","state","categories"]]
business_info.to_csv("./dataset/business_info.csv",index=False)

## Clean categories

In [7]:
# string-->list + remove space + remove Restaurants
rests5["categories"] = \
rests5["categories"].apply(lambda s:[ss.strip() for ss in s.split(",") if ss.strip() != "Restaurants"])

In [9]:
rests5["categories"][:5]

0                                              [Salad, Soup, Sandwiches, Delis, Cafes, Vegetarian]
1                                                                                           [Thai]
2                                             [American (New), Bakeries, Desserts, Food, Cupcakes]
3    [American (New), Food Court, Flowers & Gifts, Hotels & Travel, Gift Shops, Resorts, Shopping]
4                                                                 [Nightlife, Sandwiches, Seafood]
Name: categories, dtype: object

In [8]:
all_categ = [ss for l in list(rests5["categories"].values) for ss in l]
categ15 = pd.Series(all_categ).value_counts(ascending=False)[1:16].index #since first is food--meaningless
categ15

Index(['Nightlife', 'Bars', 'Sandwiches', 'American (Traditional)',
       'Breakfast & Brunch', 'Fast Food', 'American (New)', 'Mexican',
       'Coffee & Tea', 'Burgers', 'Pizza', 'Cafes', 'Food Trucks',
       'Event Planning & Services', 'Seafood'],
      dtype='object')

In [9]:
# Transform each element of a list-like(list/tuple/dict/set/series/array) to a row, replicating index values.
# list-like columns to long format [Salad, Soup, Sandwiches, Delis, Cafes, Vegetarian]每个值都变成单独一行(长表)
big = rests5[["business_id","categories"]].explode("categories")
big = big[big["categories"].isin(categ15)]
big["values"] = 1
big_categ = big.pivot_table(values = "values",index="business_id",columns="categories",
              aggfunc = "count",fill_value = 0)
big_categ.head()

# regard if not wrote this category:then the restaurant DOESNT belong to this category
# impute NaN with 0

categories,American (New),American (Traditional),Bars,Breakfast & Brunch,Burgers,Cafes,Coffee & Tea,Event Planning & Services,Fast Food,Food Trucks,Mexican,Nightlife,Pizza,Sandwiches,Seafood
business_id,,,,,,,,,,,,,,,
--UNNdnHRhsyFUbDgumdtQ,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
-0OWS89ebRdvzOQkjptyEw,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
-0p-JeIeAY_u6NEEUMgGNg,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
-1ShItlulHnBsoOQWnblzw,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
-1XOUWwxeSAVdtuXuP9twg,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [12]:
rests5.shape

(12658, 14)

In [13]:
big_categ.shape

(9816, 15)

In [10]:
# Some restaurants' categories are NOT in top15 tags
# gets filtered out in "isin" command
# therefore left join with all restaurants to keep them

big_categ.reset_index(inplace=True)
categ_full = rests5.merge(big_categ,how="left",on="business_id")
categ_full = categ_full.fillna(0) #此时的NaN只有可能来自于categories 因为其他已经drop了

In [80]:
categ_full.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,American (New),American (Traditional),Bars,Breakfast & Brunch,Burgers,Cafes,Chinese,Coffee & Tea,Event Planning & Services,Fast Food,Mexican,Nightlife,Pizza,Sandwiches,Seafood
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAttire': 'u'casual'', 'GoodForKids': 'True', 'BikeParking': 'False', 'OutdoorSeating': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'Caters': 'True', 'RestaurantsReservations': 'False', 'RestaurantsDelivery': 'False', 'HasTV': 'False', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'NoiseLevel': 'u'average'', 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '2', 'WiFi': 'u'free'', 'BusinessParking': '{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'Alcohol': 'u'beer_and_wine'', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': True}'}","[Salad, Soup, Sandwiches, Delis, Cafes, Vegetarian]","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', 'Wednesday': '5:0-18:0', 'Thursday': '5:0-18:0', 'Friday': '5:0-18:0', 'Saturday': '5:0-18:0', 'Sunday': '5:0-18:0'}",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Clean attribute
- JSON format
- expand each key to columns

In [11]:
attr_df = pd.json_normalize(rests5.loc[:,"attributes"])
attr_df.head(1)

,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Ambience,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,RestaurantsGoodForGroups,BusinessAcceptsCreditCards,NoiseLevel,ByAppointmentOnly,RestaurantsPriceRange2,WiFi,BusinessParking,Alcohol,GoodForMeal,DogsAllowed,CoatCheck,GoodForDancing,RestaurantsTableService,HappyHour,Music,WheelchairAccessible,Smoking,BestNights,BusinessAcceptsBitcoin,Corkage,BYOB,BYOBCorkage,DriveThru,RestaurantsCounterService,DietaryRestrictions,AgesAllowed,Open24Hours,AcceptsInsurance
0,True,u'casual',True,False,False,"{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}",True,False,False,False,False,True,u'average',False,2,u'free',"{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}",u'beer_and_wine',"{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': True}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# look at missing values for each column after expand keys to columns
nan_mean = attr_df.isnull().mean().sort_values(ascending=False)
nan_mean

AcceptsInsurance              0.999605
RestaurantsCounterService     0.999210
AgesAllowed                   0.999131
Open24Hours                   0.998894
DietaryRestrictions           0.998262
BYOBCorkage                   0.933481
Smoking                       0.899273
GoodForDancing                0.892874
Corkage                       0.884500
CoatCheck                     0.881972
BYOB                          0.875415
ByAppointmentOnly             0.872808
BestNights                    0.865224
DriveThru                     0.862775
Music                         0.841286
BusinessAcceptsBitcoin        0.825170
HappyHour                     0.614868
DogsAllowed                   0.610207
WheelchairAccessible          0.606731
RestaurantsTableService       0.478433
NoiseLevel                    0.276110
GoodForMeal                   0.259283
RestaurantsAttire             0.240322
Caters                        0.210381
BusinessAcceptsCreditCards    0.203034
GoodForKids              

In [13]:
# drop columns if they have >30% of missing values
dropped = nan_mean[nan_mean > 0.3].index
attr_df = attr_df.drop(columns=dropped)

In [13]:
len(dropped) #drop 20 columns

20

### Clean each subcategory

In [14]:
attr_fill = attr_df.fillna(attr_df.mode().iloc[0])

In [15]:
for col in attr_fill.columns:
    print(attr_fill[col].value_counts(dropna=False))

True     11636
False      607
None       415
Name: RestaurantsTakeOut, dtype: int64
u'casual'    8180
'casual'     4251
u'dressy'     118
'dressy'       94
'formal'        5
None            5
u'formal'       5
Name: RestaurantsAttire, dtype: int64
True     10931
False     1723
None         4
Name: GoodForKids, dtype: int64
True     10720
False     1936
None         2
Name: BikeParking, dtype: int64
True     7354
False    4277
None     1027
Name: OutdoorSeating, dtype: int64
{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': False}    3150
{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}     1189
{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}      908
{'touristy': F

In [16]:
# string type:remove "u" and "''" single quotation
obj_c = ["RestaurantsAttire",
        "NoiseLevel",
        "Alcohol",
        "WiFi"]

attr_fill[obj_c] = attr_fill[obj_c].applymap(lambda s:re.sub(r"^[u\']+","",s))\
.applymap(lambda s:re.sub(r"[\']+","",s))

`None` replace as `False`：

In [17]:
type1 = ["RestaurantsTakeOut",
"BikeParking",
"OutdoorSeating",
"Caters",
"RestaurantsReservations",
"RestaurantsDelivery",
"HasTV",
"RestaurantsGoodForGroups",
"WiFi",
"Alcohol"]
attr_fill[type1] = attr_fill[type1].replace("None","False")

Replace `None` with mode from each column:

In [18]:
type2 = [
"GoodForKids",
"NoiseLevel",
"RestaurantsPriceRange2"]
for c in type2:
    attr_fill[c] = \
    attr_fill[c].replace("None",attr_fill[c].mode()[0])

Replace `False` with mode from each column:

In [19]:
attr_fill["RestaurantsAttire"] = \
attr_fill["RestaurantsAttire"].replace("False",attr_fill["RestaurantsAttire"].mode()[0])

In [20]:
# boolean type
bools = [
    "RestaurantsTakeOut",
    "GoodForKids",
    "BikeParking",
    "OutdoorSeating",
    "Caters",
    "RestaurantsReservations",
    "RestaurantsDelivery",
    "HasTV",
    "RestaurantsGoodForGroups",
    "BusinessAcceptsCreditCards"
]
attr_fill[bools] = attr_fill[bools].replace({"True":1,"False":0})

Columns that need to be cleaned separately
- since they have different cleaning logic

In [23]:
attr_fill["Alcohol"] = \
attr_fill["Alcohol"].replace(["none","False"],0).replace(["full_bar","beer_and_wine"],1)

In [24]:
attr_fill["WiFi"] = \
attr_fill["WiFi"].replace(["free","paid"],1).replace(["no","False"],0)

In [25]:
attr_fill["RestaurantsPriceRange2"] = attr_fill["RestaurantsPriceRange2"].astype("int") 

In [26]:
attr_fill.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12658 entries, 0 to 12657
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   RestaurantsTakeOut          12658 non-null  int64 
 1   RestaurantsAttire           12658 non-null  object
 2   GoodForKids                 12658 non-null  int64 
 3   BikeParking                 12658 non-null  int64 
 4   OutdoorSeating              12658 non-null  int64 
 5   Ambience                    12658 non-null  object
 6   Caters                      12658 non-null  int64 
 7   RestaurantsReservations     12658 non-null  int64 
 8   RestaurantsDelivery         12658 non-null  int64 
 9   HasTV                       12658 non-null  int64 
 10  RestaurantsGoodForGroups    12658 non-null  int64 
 11  BusinessAcceptsCreditCards  12658 non-null  int64 
 12  NoiseLevel                  12658 non-null  object
 13  RestaurantsPriceRange2      12658 non-null  in

In [27]:
mapping = {"quiet":0,"average":1,"False":1,"loud":2,"very_loud":3}
attr_fill["NoiseLevel"] = attr_fill["NoiseLevel"].map(mapping)

In [110]:
attr_fill.columns

Index(['RestaurantsTakeOut', 'RestaurantsAttire', 'GoodForKids', 'BikeParking',
       'OutdoorSeating', 'Ambience', 'Caters', 'RestaurantsReservations',
       'RestaurantsDelivery', 'HasTV', 'RestaurantsGoodForGroups',
       'BusinessAcceptsCreditCards', 'NoiseLevel', 'RestaurantsPriceRange2',
       'WiFi', 'BusinessParking', 'Alcohol', 'GoodForMeal'],
      dtype='object')

In [28]:
keep = ['RestaurantsTakeOut', 'RestaurantsAttire', 'GoodForKids', 'BikeParking',
       'OutdoorSeating', 'Caters', 'RestaurantsReservations',
       'RestaurantsDelivery', 'HasTV', 'RestaurantsGoodForGroups',
       'BusinessAcceptsCreditCards', 'NoiseLevel', 'RestaurantsPriceRange2',
       'WiFi','Alcohol']
attr_keep = attr_fill.loc[:,keep]

In [29]:
for c in keep:
    print(attr_keep[c].value_counts())

1    11636
0     1022
Name: RestaurantsTakeOut, dtype: int64
casual    12431
dressy      212
formal       10
None          5
Name: RestaurantsAttire, dtype: int64
1    10935
0     1723
Name: GoodForKids, dtype: int64
1    10720
0     1938
Name: BikeParking, dtype: int64
1    7354
0    5304
Name: OutdoorSeating, dtype: int64
1    8052
0    4606
Name: Caters, dtype: int64
0    9126
1    3532
Name: RestaurantsReservations, dtype: int64
1    8430
0    4228
Name: RestaurantsDelivery, dtype: int64
1    9273
0    3385
Name: HasTV, dtype: int64
1    11197
0     1461
Name: RestaurantsGoodForGroups, dtype: int64
1    12502
0      156
Name: BusinessAcceptsCreditCards, dtype: int64
1    10461
0     1299
2      721
3      177
Name: NoiseLevel, dtype: int64
2    7331
1    4828
3     424
4      75
Name: RestaurantsPriceRange2, dtype: int64
1    8566
0    4092
Name: WiFi, dtype: int64
0    6617
1    6041
Name: Alcohol, dtype: int64


### BusinessParking

In [114]:
attr_fill["BusinessParking"][0]

"{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"

In [105]:
print(type(attr_fill["BusinessParking"][0])) 
#str type cannot use json_normalize directly! must be dict/JSON

eval(attr_fill["BusinessParking"][0]) #converting string to json #dict->json:loads
# pd.json_normalize(eval(attr_df["BusinessParking"][0]))

<class 'str'>


{'garage': True,
 'street': False,
 'validated': False,
 'lot': False,
 'valet': False}

In [30]:
attr_fill["BusinessParking"] = \
attr_fill["BusinessParking"].replace("False",attr_fill["BusinessParking"].mode()[0])\
.replace("{}",attr_fill["BusinessParking"].mode()[0]).replace("None",attr_fill["BusinessParking"].mode()[0])

In [31]:
# string-->json
parking_s = attr_fill["BusinessParking"].apply(lambda s:eval(s)) 
parking_df = pd.json_normalize(parking_s)

In [128]:
# check missing values
parking_df.isnull().sum() 

garage       385
street       426
validated    324
lot          372
valet          0
dtype: int64

In [32]:
# impute missing values
parking_df = parking_df.fillna(False)
parking_df.isnull().sum()

garage       0
street       0
validated    0
lot          0
valet        0
dtype: int64

### Ambience

In [33]:
# "False"-->treat as NOT have any ambience type
default = "{'touristy': False,\
             'hipster': False,\
             'romantic': False,\
             'divey': False,\
             'intimate': False,\
             'trendy': False,\
             'upscale': False,\
             'classy': False,\
             'casual': False}"
attr_fill["Ambience"] = \
attr_fill["Ambience"].replace(["False","None"],default)

# missing--impute with mode
attr_fill["Ambience"] = \
attr_fill["Ambience"].fillna(attr_fill["Ambience"].mode()[0])

In [34]:
amb_s = attr_fill["Ambience"].apply(lambda s:eval(s)) #string-->json format in order to use json_normalize
amb_df = pd.json_normalize(list(amb_s.values))

In [35]:
amb_df = amb_df.fillna(False)

In [169]:
amb_df.isnull().sum()

romantic    0
intimate    0
touristy    0
hipster     0
divey       0
classy      0
trendy      0
upscale     0
casual      0
dtype: int64

### GoodForMeal

In [36]:
# GoodForMeal
default = "{'dessert': False,\
 'latenight': False,\
 'lunch': False,\
 'dinner': False,\
 'brunch': False,\
 'breakfast': False}"
attr_fill["GoodForMeal"] = \
attr_fill["GoodForMeal"].replace(["False","None"],default)

In [37]:
meal_s = attr_fill["GoodForMeal"].apply(lambda s:eval(s))

meal_df = pd.json_normalize(meal_s)

In [38]:
meal_df = meal_df.fillna(False)

### Combine attributes

In [39]:
attr_df_final = pd.concat([attr_keep,parking_df,amb_df,meal_df],axis=1)
attr_df_final.shape

(12658, 35)

In [40]:
# boolean columns convert to numeric columns
bools = attr_df_final.select_dtypes("bool").columns
attr_df_final[bools] = attr_df_final[bools].astype("int")

In [36]:
attr_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12658 entries, 0 to 12657
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   RestaurantsTakeOut          12658 non-null  int64 
 1   RestaurantsAttire           12658 non-null  object
 2   GoodForKids                 12658 non-null  int64 
 3   BikeParking                 12658 non-null  int64 
 4   OutdoorSeating              12658 non-null  int64 
 5   Caters                      12658 non-null  int64 
 6   RestaurantsReservations     12658 non-null  int64 
 7   RestaurantsDelivery         12658 non-null  int64 
 8   HasTV                       12658 non-null  int64 
 9   RestaurantsGoodForGroups    12658 non-null  int64 
 10  BusinessAcceptsCreditCards  12658 non-null  int64 
 11  NoiseLevel                  12658 non-null  int64 
 12  RestaurantsPriceRange2      12658 non-null  int32 
 13  WiFi                        12658 non-null  ob

## Clean hour

In [42]:
hour_df = pd.json_normalize(rests5["hours"]) # dict-->flat table

In [140]:
hour_df.head(10)

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,5:0-18:0,5:0-17:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0
1,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0,17:0-21:0
2,NaN,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0,NaN
3,0:0-0:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0,7:0-22:0
4,6:30-22:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-22:0,6:30-20:0
5,6:0-21:0,6:0-21:0,6:0-21:0,6:0-21:0,6:0-21:0,6:0-21:0,6:0-21:0
6,7:0-23:0,7:0-23:0,7:0-23:0,7:0-23:0,7:0-0:0,7:0-0:0,7:0-23:0
7,0:0-0:0,11:0-22:0,11:0-22:0,16:30-22:0,11:0-22:0,11:0-22:0,11:0-22:0
8,10:0-17:0,10:0-17:0,10:0-17:0,10:0-17:0,10:0-17:0,10:0-16:0,10:0-16:0
9,6:30-1:0,6:30-1:0,6:30-1:0,6:30-1:0,6:30-1:0,6:30-1:0,6:30-1:0


- New Variables：
    - `open_days` how many days a restaurant opens per week（non-null values in a row）
    - `open_hours` how many hours a restaurant opens per week

In [43]:
open_s = (~hour_df.isnull()).sum(axis=1) #how many non-null values in a row

In [45]:
# total hours per week
def count_hours(row):
    """
    row:dict type
    """
    week_hours = 0
    for val in row.values(): #extract values from dict:everyday opening hours
        opent,closet = val.split("-")
        opent = datetime.strptime(opent,"%H:%M") #5:0--input only has H & M part
        closet = datetime.strptime(closet,"%H:%M") #18:0
        duration = closet - opent 
        hour = duration.seconds/3600
        week_hours += hour
    return week_hours

In [46]:
hour_s = rests5["hours"].apply(count_hours)

In [47]:
hour_df_final = pd.concat([open_s,hour_s],axis=1)
hour_df_final = hour_df_final.rename(columns={0:"open_days","hours":"open_hours"})

## Final Concatenation

In [48]:
# First reset index before concatenating
categ_full.reset_index(drop=True,inplace=True)
attr_df_final.reset_index(drop=True,inplace=True)
hour_df_final.reset_index(drop=True,inplace=True)
rests_final = pd.concat([categ_full,attr_df_final,hour_df_final],axis=1)

In [45]:
rests_final.shape

(12658, 66)

In [189]:
rests_final.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,American (New),American (Traditional),Bars,Breakfast & Brunch,Burgers,Cafes,Coffee & Tea,Event Planning & Services,Fast Food,Food Trucks,Mexican,Nightlife,Pizza,Sandwiches,Seafood,RestaurantsTakeOut,RestaurantsAttire,GoodForKids,BikeParking,OutdoorSeating,Caters,RestaurantsReservations,RestaurantsDelivery,HasTV,RestaurantsGoodForGroups,BusinessAcceptsCreditCards,NoiseLevel,RestaurantsPriceRange2,WiFi,Alcohol,garage,street,validated,lot,valet,romantic,intimate,touristy,hipster,divey,classy,trendy,upscale,casual,dessert,latenight,lunch,dinner,brunch,breakfast,open_days,open_hours
0,tCbdrRPZA0oiIYSmHG3J0w,Flying Elephants at PDX,7000 NE Airport Way,Portland,OR,97218,45.588906,-122.593331,4.0,126,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsAttire': 'u'casual'', 'GoodForKids': 'True', 'BikeParking': 'False', 'OutdoorSeating': 'False', 'Ambience': '{'romantic': False, 'intimate': False, 'touristy': False, 'hipster': False, 'divey': False, 'classy': False, 'trendy': False, 'upscale': False, 'casual': True}', 'Caters': 'True', 'RestaurantsReservations': 'False', 'RestaurantsDelivery': 'False', 'HasTV': 'False', 'RestaurantsGoodForGroups': 'False', 'BusinessAcceptsCreditCards': 'True', 'NoiseLevel': 'u'average'', 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '2', 'WiFi': 'u'free'', 'BusinessParking': '{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}', 'Alcohol': 'u'beer_and_wine'', 'GoodForMeal': '{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': False, 'breakfast': True}'}","[Salad, Soup, Sandwiches, Delis, Cafes, Vegetarian]","{'Monday': '5:0-18:0', 'Tuesday': '5:0-17:0', 'Wednesday': '5:0-18:0', 'Thursday': '5:0-18:0', 'Friday': '5:0-18:0', 'Saturday': '5:0-18:0', 'Sunday': '5:0-18:0'}",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,casual,1,0,0,1,0,0,0,0,1,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,7,90.0


In [49]:
# drop unnecessary columns
# since postal_code cannot be treated as number
dropped = ["business_id","address","postal_code","attributes","name","hours","categories"] 
rests_final = rests_final.drop(dropped,axis=1)

In [50]:
rests_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12658 entries, 0 to 12657
Data columns (total 59 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   city                        12658 non-null  object 
 1   state                       12658 non-null  object 
 2   latitude                    12658 non-null  float64
 3   longitude                   12658 non-null  float64
 4   stars                       12658 non-null  float64
 5   review_count                12658 non-null  int64  
 6   is_open                     12658 non-null  int64  
 7   American (New)              12658 non-null  float64
 8   American (Traditional)      12658 non-null  float64
 9   Bars                        12658 non-null  float64
 10  Breakfast & Brunch          12658 non-null  float64
 11  Burgers                     12658 non-null  float64
 12  Cafes                       12658 non-null  float64
 13  Coffee & Tea                126

In [51]:
# get_dummies for object columns
obj_c = rests_final.select_dtypes("O").columns
rests_final = pd.get_dummies(rests_final,columns=obj_c)

In [52]:
rests_final.shape

(12658, 73)

## save to CSV file

In [54]:
filename = "./dataset/yelp_business_cleaned.csv"
rests_final.to_csv(filename,index=False)